In [24]:
!pip3 install -U selenium

In [25]:
!pip3 install webdriver_manager

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# initialize the browser
browser = webdriver.Chrome(ChromeDriverManager().install())
url = "https://coinmarketcap.com/currencies/verasity/historical-data/"
browser.get(url)
time.sleep(1)

# make sure the entire page is loaded in
browser.execute_script("window.scrollTo(0,1000)")
time.sleep(0.1)
browser.execute_script("window.scrollTo(0,3600)")
time.sleep(0.1)

# find a close the cookie notification
elem_cookie_close = browser.find_element_by_xpath('/html/body/div/div[1]/div[2]/div[2]/div[2]')
elem_cookie_close.click()
time.sleep(0.1)

# locate the load more button
elem_button = browser.find_element_by_xpath('/html/body/div/div[1]/div[1]/div[2]/div/div[3]/div/div/p[1]/button')
coordinates = elem_button.location_once_scrolled_into_view
browser.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
time.sleep(0.1)

# click the load more button x amount of times to load more price data
for x in range(8):
    elem_button.click()
    coordinates = elem_button.location_once_scrolled_into_view
    browser.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
    time.sleep(0.2)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Get LATEST driver version for 91.0.4472
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Vincent\.wdm\drivers\chromedriver\win32\91.0.4472.101]


In [3]:
# find the body of the price data
elem_tbody = browser.find_element_by_tag_name('tbody')

In [4]:
# find each row in the table
elems_tr = elem_tbody.find_elements_by_tag_name('tr')
len(elems_tr)

182

In [5]:
# loop through each row and print as string
for tr in elems_tr:
    elems_td = tr.find_elements_by_tag_name('td')
    print('date={date} open={open_price} close={close_price} market_cap={market_cap}'.format(
        date=elems_td[0].text,
        open_price=elems_td[1].text,
        close_price=elems_td[4].text,
        market_cap=elems_td[6].text))

date=Jun 21, 2021 open=$0.01 close=$0.01 market_cap=$45,579,077
date=Jun 20, 2021 open=$0.01 close=$0.01 market_cap=$55,565,523
date=Jun 19, 2021 open=$0.01 close=$0.01 market_cap=$55,244,822
date=Jun 18, 2021 open=$0.02 close=$0.01 market_cap=$54,775,540
date=Jun 17, 2021 open=$0.02 close=$0.02 market_cap=$58,953,596
date=Jun 16, 2021 open=$0.02 close=$0.02 market_cap=$59,299,637
date=Jun 15, 2021 open=$0.02 close=$0.02 market_cap=$62,948,609
date=Jun 14, 2021 open=$0.02 close=$0.02 market_cap=$63,904,066
date=Jun 13, 2021 open=$0.01 close=$0.02 market_cap=$65,023,247
date=Jun 12, 2021 open=$0.02 close=$0.01 market_cap=$55,501,512
date=Jun 11, 2021 open=$0.02 close=$0.02 market_cap=$58,772,453
date=Jun 10, 2021 open=$0.02 close=$0.02 market_cap=$64,512,933
date=Jun 09, 2021 open=$0.01 close=$0.02 market_cap=$64,740,512
date=Jun 08, 2021 open=$0.02 close=$0.01 market_cap=$55,909,977
date=Jun 07, 2021 open=$0.02 close=$0.02 market_cap=$59,661,987
date=Jun 06, 2021 open=$0.02 close=$0.02

In [6]:
# put the data in a pandas dataframe
import pandas as pd
from datetime import date, timedelta, datetime

dataframe_list = []

for tr in elems_tr:
    elems_td = tr.find_elements_by_tag_name('td')
    
    # fix the format of the date
    input_date = elems_td[0].text
    input_date_object = datetime.strptime(input_date,'%b %d, %Y')
    formatted_date = input_date_object.strftime('%Y-%m-%d')
    
    dataframe_list.append({'date':formatted_date, 'open':elems_td[1].text, 'close':elems_td[4].text, 'market_cap':elems_td[6].text})
    
price_data = pd.DataFrame(data=dataframe_list)
price_data.to_csv('price-data/vra-price.csv', index=False) 
browser.close()
print("Finished scraping price data")

Finished scraping price data


In [9]:
df = price_data.copy()
df['market_cap'] = df['market_cap'].apply(lambda x: x.replace('$', '').replace(',', '')
                                if isinstance(x, str) else x).astype(float)
df['market_cap'] = df['market_cap'].apply(lambda x: x / 1000000).astype(float)
df['open'] = df['open'].apply(lambda x: x.replace('$', '').replace(',', '')
                                if isinstance(x, str) else x).astype(float)
df['close'] = df['close'].apply(lambda x: x.replace('$', '').replace(',', '')
                                if isinstance(x, str) else x).astype(float)
df.to_csv('price-data/vra-price.csv', index=False) 
print("Finished cleaning data")

Finished cleaning data
